In [23]:
import numpy as np
import sys, os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import measure
import cv2
from skimage import morphology
from skimage.morphology import disk, square, dilation, erosion
from skimage import data
from skimage.feature import Cascade
from EyeCenterLocator import EyeCenterLocator
from FaceAligner import FaceAligner
import imutils
from scipy import signal
from skimage import filters
from skimage import feature
import skimage.io as io
from scipy import fftpack
from scipy.signal import convolve2d
from skimage.util import random_noise
from skimage.exposure import rescale_intensity
from PIL import Image
import requests

In [24]:
kEyePercentTop = 25
kEyePercentSide = 8
kEyePercentHeight = 30
kEyePercentWidth = 35

eye_percent_height = 28
eye_percent_width = 18

mouth_percent_height = 33
mouth_percent_width = 25

In [25]:
def contrastStretching(original):
    img = original.copy()
    xp = [0, 64, 128, 192, 255]
    fp = [0, 16, 128, 240, 255]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    img = cv2.LUT(img, table)
    return img

In [26]:
def skinDetection(sourceImage):
    min_YCrCb = np.array([0,133,77],np.uint8)
    max_YCrCb = np.array([255,173,127],np.uint8)

    # Convert image to YCrCb
    imageYCrCb = cv2.cvtColor(sourceImage,cv2.COLOR_RGB2YCR_CB)

    # Find region with skin tone in YCrCb image
    skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

    # Do contour detection on skin region
    contours, hierarchy = cv2.findContours(skinRegion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contour on the source image
    max_contour_area = -1
    max_contour_index = -1
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        if area > max_contour_area:
            max_contour_index = i
            max_contour_area = area
    sourceImage = np.zeros(sourceImage.shape).astype(sourceImage.dtype)
    if max_contour_index > -1: 
        cv2.drawContours(sourceImage, contours, max_contour_index, (255, 255, 255), -1)
    sourceImage = cv2.cvtColor(sourceImage,cv2.COLOR_RGB2GRAY)
    return sourceImage

In [27]:
def getEyesCenter(face_BGR):
    
    eyeCenterLocator = EyeCenterLocator()
    img_GRAY = cv2.cvtColor(face_BGR, cv2.COLOR_BGR2GRAY)
    
    eye_region_width = face_BGR.shape[0] * (kEyePercentWidth/100.0)
    eye_region_height = face_BGR.shape[0] * (kEyePercentHeight/100.0)
    eye_region_top = face_BGR.shape[1] * (kEyePercentTop/100.0)
    
    global leftEyeRegion
    global rightEyeRegion
    leftEyeRegion = (1,1,1,1)
    rightEyeRegion = (1,1,1,1)

    leftEyeRegion = int(face_BGR.shape[0]*(kEyePercentSide/100.0)), int(eye_region_top), int(eye_region_width), int(eye_region_height)
    rightEyeRegion = int(face_BGR.shape[0] - eye_region_width - face_BGR.shape[0]*(kEyePercentSide/100.0)), int(eye_region_top),int(eye_region_width),int(eye_region_height)
    
    x1, y1, w1, h1 = (leftEyeRegion)
    region = img_GRAY[int(y1) :int(y1) + int(h1) , int(x1) :int(x1) + int(w1)]
    leftEyeCenter = eyeCenterLocator.locate(region)
    
    x2, y2, w2, h2 = (rightEyeRegion)
    region = img_GRAY[int(y2) :int(y2) + int(h2) , int(x2) :int(x2) + int(w2)]
    rightEyeCenter = eyeCenterLocator.locate(region)
    
    return (int(leftEyeCenter[1])+int(x1) , int(leftEyeCenter[0])+int(y1)), (int(rightEyeCenter[1])+int(x2), int(rightEyeCenter[0])+int(y2))

In [39]:
def HarrisCorners(image_BGR):
    if image_BGR.shape == (0,0) or image_BGR is None or len(image_BGR)==0:
        return (0,0),(0,0)
    
    gray = cv2.cvtColor(image_BGR,cv2.COLOR_BGR2GRAY)
    ret,threshed = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    threshed = cv2.erode(threshed,None,2)
    corner = cv2.cornerHarris(threshed, 2, 3, 0.04)
    
    maxLeftCorner = image_BGR.shape[1]
    maxRieghtCorner = 0
    
    maxDistance = 0
    currentDistance = maxRieghtCorner - maxLeftCorner
    
    maxLeftCornerPosition =(0,0)
    maxRightCornerPosition =(0,0)
    
    for i in range(0, image_BGR.shape[0]):
         for j in range(0, image_BGR.shape[1]):
            if corner[i, j] > 0:
                if maxLeftCorner > j:
                    maxLeftCorner = j
                    maxLeftCornerPosition = (j,i) 
                        
                if maxRieghtCorner < j:
                    maxRieghtCorner = j
                    maxRightCornerPosition = (j,i)                   
                    currentDistance = maxRieghtCorner-maxLeftCorner
                    
                if maxDistance<currentDistance:
                    maxDistance = maxRieghtCorner-maxLeftCorner
    return maxLeftCornerPosition, maxRightCornerPosition

In [29]:
def getEyeCorners(face_BGR, EyeCenter, left):
    
    global leftEyeCRegion
    global rightEyeCRegion
    
    eye_height = int(face_BGR.shape[0]*(eye_percent_height/100.0))
    eye_width = int(face_BGR.shape[1]*(eye_percent_width/100.0))
    
    top_corner = (int(EyeCenter[0]-eye_height//2), int(EyeCenter[1]-eye_width//2))
    bootom_corner = (int(EyeCenter[0]+eye_height//2), int(EyeCenter[1]+eye_width//2))
    
    if(left == True):
        leftEyeCRegion = (top_corner[0], top_corner[1], eye_height, eye_width)
    else:
        rightEyeCRegion = (top_corner[0], top_corner[1], eye_height, eye_width)
        
    eye_ROI = face_BGR[top_corner[1]:bootom_corner[1], top_corner[0]:bootom_corner[0]]
    left_corner, right_corner = HarrisCorners(eye_ROI)
        
    left_corner = (left_corner[0]+top_corner[0], left_corner[1]+top_corner[1])
    right_corner = (right_corner[0]+top_corner[0], right_corner[1]+top_corner[1])
    
    return left_corner, right_corner

In [30]:
def getEyesCorners(face_BGR, leftEyeCenter, rightEyeCenter):
    global leftEyeCRegion
    global rightEyeCRegion
    leftEyeCRegion = (1,1,1,1)
    rightEyeCRegion = (1,1,1,1)
    left_eye_left_corner, left_eye_right_corner = getEyeCorners(face_BGR, leftEyeCenter, True)
    right_eye_left_corner, right_eye_right_corner = getEyeCorners(face_BGR, rightEyeCenter, False)
    return left_eye_left_corner, left_eye_right_corner, right_eye_left_corner, right_eye_right_corner

In [31]:
def getMouthCorners(face_BGR):
    global mouthRegion
    mouthRegion = (1,1,1,1)
    
    # Center of lower half
    center = (int((face_BGR.shape[0]-1-0)//2), int(face_BGR.shape[0]//2 + face_BGR.shape[0]//4))
    #low_center = (center[0], face_BGR.shape[0]-1)
    
    mouth_height = int(face_BGR.shape[0]*(mouth_percent_height/100.0))
    mouth_width = int(face_BGR.shape[1]*(mouth_percent_width/100.0))
    
    top_corner = (center[0]-mouth_height//2, center[1]-mouth_width//2)
    bottom_corner = (center[0]+mouth_height//2, center[1]+mouth_width//2)
    
    mouthRegion = (top_corner[0], top_corner[1], mouth_height, mouth_width)
    
    mouth_ROI = face_BGR[top_corner[1]:bottom_corner[1], top_corner[0]:bottom_corner[0]]
    cv2.imshow('mouth_ROI', mouth_ROI)
    left_corner, right_corner = HarrisCorners(mouth_ROI)
    
    left_corner = (left_corner[0]+top_corner[0], left_corner[1]+top_corner[1])
    right_corner = (right_corner[0]+top_corner[0],right_corner[1]+top_corner[1])
    
    return left_corner, right_corner

In [32]:
def GetMouthMap(image_RGB):
    # transform the image to the YCbCr space
    image_YCrCb = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2YCR_CB)
    
    # extracting YCbCr space components
    Y = image_YCrCb[:,:,0]
    Cr = np.asarray(image_YCrCb[:,:,1], np.int16)
    Cb = np.asarray(image_YCrCb[:,:,2], np.int16)
    
    skin = skinDetection(image_RGB)
    
    # calculating the MouthMap
    n = np.count_nonzero(skin==255)
        
    eta = 0
    
    try:
        v_1 = np.power(Cr[skin==255],2)
        v_1 =  v_1/(np.amax(v_1)/255.0)

        v_2 = np.divide(Cr[skin==255],Cb[skin==255])
        v_2 =  v_2/(np.amax(v_2)/255.0)

        eta = 0.95 * np.sum(v_1)/np.sum(v_2)
    except ValueError:  #raised if `v_1` is empty or `v_2`
        pass
    
    val_1 = np.power(Cr,2)
    val_1 = val_1/(np.amax(val_1)/255.0)

    val_2 = Cr/Cb
    val_2 = val_2/(np.amax(val_2)/255.0)

    MouthMap = val_1 * np.power(val_1 - eta * val_2, 2)
    MouthMap = MouthMap/(np.amax(MouthMap)/255.0)
    
    mask = disk(10)
    MouthMap = dilation(MouthMap, selem=mask)
    
    MouthMap = cv2.bitwise_and(MouthMap,  MouthMap,mask =skin)
    
    MouthMap = np.asarray(MouthMap, dtype='uint8')
    
    ret2,MouthMap = cv2.threshold(MouthMap,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return MouthMap

In [33]:
def MouthMapROI(image_RGB):
    mouth_ROI = face[2*face.shape[0]//3:face.shape[0],face.shape[1]//3:2*face.shape[1]//3]
    start_x = w//3
    start_y = 2*h//3
    mouth = GetMouthMap(mouth_ROI)
    contours, hierarchy = cv2.findContours(mouth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    l = len(contours)
    #for contour in contours:
    x_c = 0
    y_c = 0
    w_c = 0
    h_c = 0
    if len(contours) > 0:
        x_c,y_c,w_c,h_c = cv2.boundingRect(contours[-1])
        x_c += (start_x)
        y_c += (start_y)
    return x_c,y_c,w_c,h_c 

In [34]:
def preprocessing(img_BGR):
    kernel = np.ones((3,3),np.float32)/9
    img_BGR = cv2.filter2D(img_BGR,-1,kernel)
    img_BGR = contrastStretching(img_BGR)
    img_BGR[:,:,0] = cv2.equalizeHist(img_BGR[:,:,0])
    img_BGR[:,:,1] = cv2.equalizeHist(img_BGR[:,:,1])
    img_BGR[:,:,2] = cv2.equalizeHist(img_BGR[:,:,2])
    #img_BGR = contrastStretching(img_BGR)
    return img_BGR

In [35]:
def cropImage(image, p_width, p_hieght):
    width = image.shape[0]
    height = image.shape[1]
    left = int((width - width*p_width)//2)
    top = int((height - height*p_hieght)//2)
    right = int((width +  width*p_width)//2)
    bottom = int((height + height*p_hieght)//2)
    
    return image[left:right, top:bottom]

In [36]:
def drawLandmarks(img_BGR):
    #img_BGR = preprocessing(img_BGR)
    #img_BGR = cv2.resize(img_BGR, (480, 480))
    imageAligned = img_BGR
    img = img_BGR.copy()
    
    img_GRAY = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img_GRAY)
    fa = FaceAligner(desiredFaceWidth=256)
    if len(faces) == 0:
        cv2.destroyAllWindows()
        webcam.release()
        sys.exit()
    for (x, y, w, h) in faces:
        top = (x, y)
        face_rect = (x, y, w, h)
        
        face = img_BGR[y :y + h , x :x + w ]
        face = cropImage(face, 0.9, 0.8)
        faceAligned = face 
        #face = cv2.resize(face, (256, 256))
        
        try:
            (leftEyeCenter, rightEyeCenter) = getEyesCenter(face)
            #(LEyeLCorner, LEyeRCorner, REyeLCorner, REyeRCorner) = getEyesCorners(face, leftEyeCenter, rightEyeCenter)
            #(LMouthCorner, RMouthCorner) = getMouthCorners(face)
        except:
            continue
        
        imageAligned = fa.align(img_BGR, (rightEyeCenter[0]+x, rightEyeCenter[1]+y), (leftEyeCenter[0]+x, leftEyeCenter[1]+y))
        
        croped_face = face_cascade.detectMultiScale(cv2.cvtColor(imageAligned, cv2.COLOR_BGR2GRAY), 1.3, 4)
        if len(croped_face)>0:
            (xA, yA, wA, hA) = croped_face[0]
            faceAligned = imageAligned[yA :yA + hA , xA :xA + wA]
            
           
            try:
                (leftEyeCenter, rightEyeCenter) = getEyesCenter(faceAligned)
                (LEyeLCorner, LEyeRCorner, REyeLCorner, REyeRCorner) = getEyesCorners(faceAligned, leftEyeCenter, rightEyeCenter)
                (LMouthCorner, RMouthCorner) = getMouthCorners(faceAligned)
            except:
                continue
            
            img_BGR = cv2.circle(img_BGR, (leftEyeCenter[0]+x, leftEyeCenter[1]+y), 1, (0,0,255), -1)
            img_BGR = cv2.circle(img_BGR, (rightEyeCenter[0]+x, rightEyeCenter[1]+y), 1, (0,0,255), -1)

            faceAligned = cv2.rectangle(faceAligned, (leftEyeRegion[0], leftEyeRegion[1]), (leftEyeRegion[0] + leftEyeRegion[2], leftEyeRegion[1] + leftEyeRegion[3]), (255,0,255), 1)
            faceAligned = cv2.rectangle(faceAligned, (rightEyeRegion[0], rightEyeRegion[1]), (rightEyeRegion[0] + rightEyeRegion[2], rightEyeRegion[1] + rightEyeRegion[3]), (255,0,255), 1)
            
            faceAligned = cv2.circle(faceAligned, (leftEyeCenter[0], leftEyeCenter[1]), 2, (255,0,255), -1)
            faceAligned = cv2.circle(faceAligned, (rightEyeCenter[0], rightEyeCenter[1]), 2, (255,0,255), -1)


            faceAligned = cv2.rectangle(faceAligned, (leftEyeCRegion[0], leftEyeCRegion[1]), (leftEyeCRegion[0] + leftEyeCRegion[2], leftEyeCRegion[1] + leftEyeCRegion[3]), (0,255,0), 1)

            faceAligned = cv2.circle(faceAligned, (LEyeLCorner[0], LEyeLCorner[1]), 2, (0,255,0), -1)
            faceAligned = cv2.circle(faceAligned, (LEyeRCorner[0], LEyeRCorner[1]), 2, (0,255,0), -1)

            faceAligned = cv2.rectangle(faceAligned, (rightEyeCRegion[0], rightEyeCRegion[1]), (rightEyeCRegion[0] + rightEyeCRegion[2], rightEyeCRegion[1] + rightEyeCRegion[3]), (0,255,0), 1)

            faceAligned = cv2.circle(faceAligned, (REyeLCorner[0], REyeLCorner[1]), 2, (0,255,0), -1)
            faceAligned = cv2.circle(faceAligned, (REyeRCorner[0], REyeRCorner[1]), 2, (0,255,0), -1)

            faceAligned = cv2.rectangle(faceAligned, (mouthRegion[0], mouthRegion[1]), (mouthRegion[0] + mouthRegion[2], mouthRegion[1] + mouthRegion[3]), (0,100,255), 1)

            faceAligned = cv2.circle(faceAligned, (LMouthCorner[0], LMouthCorner[1]), 2, (0,100,255), -1)
            faceAligned = cv2.circle(faceAligned, (RMouthCorner[0], RMouthCorner[1]), 2, (0,100,255), -1)
        break
        
    return imageAligned, face, faceAligned

# Testing using webcam

In [38]:
haar_file = 'haarcascade_frontalface_default.xml'
webcam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(haar_file)
leftEyeCenter = (0, 0)
rightEyeCenter = (0, 0)
leftEyeRegionStart = (0, 0)
rightEyeRegionStart= (0, 0)
face_rect = (0, 0, 0, 0)
fa = FaceAligner(desiredFaceWidth=480)
while True:
    (_, img_BGR) = webcam.read()
    
    #edges = cv2.Canny(cv2.cvtColor(img_BGR,cv2.COLOR_BGR2GRAY),50,200)
    #cv2.imshow('Egdes', edges)
    
    (imageAligned, face, faceAligned) = drawLandmarks(img_BGR)
    
    cv2.imshow('Face before 2D Alignment', face)
    cv2.imshow('Face after 2D Alignment', faceAligned)
    cv2.imshow('Image after 2D Alignment', imageAligned)
    cv2.imshow('Image before 2D Alignment', img_BGR)
    
    key = cv2.waitKey(5)    
    if key == 27:
        break
    
cv2.destroyAllWindows()
webcam.release()

SystemExit: 

# Testing using stream

In [22]:
haar_file = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)
fa = FaceAligner(desiredFaceWidth=480)

leftEyeCenter = (0, 0)
rightEyeCenter = (0, 0)
leftEyeRegionStart = (0, 0)
rightEyeRegionStart= (0, 0)
face_rect = (0, 0, 0, 0)

url = "http://192.168.137.182:8080/shot.jpg"

while True:
    
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img_BGR = cv2.imdecode(img_arr, -1)
    img_BGR = imutils.resize(img_BGR, width=480, height=960)
    #cv2.imshow("Android_cam", img)
    
    (imageAligned, face, faceAligned) = drawLandmarks(img_BGR)
    
    cv2.imshow('Face before 2D Alignment', face)
    cv2.imshow('Face after 2D Alignment', faceAligned)
    cv2.imshow('Image after 2D Alignment', imageAligned)
    cv2.imshow('Image before 2D Alignment', img_BGR)
    
    key = cv2.waitKey(5)    
    if key == 27:
        break
    
cv2.destroyAllWindows()

SystemExit: 